# Поиск вырубок: разведка

Цель всех действий, которые производятся ниже -- понять, как выглядят вырубки на снимках, и какими призаками можно пользоваться, чтобы эти вырубки "поймать".

Пока я вижу следующие потенциально рабочие подходы:
 
1. Анализ временной динамики -- вырубка это резкое изменение отражательной способности. Причем изменение не любое, а определенного вида. Такой подход подразумевает, что мы можем обрабатывать каждый пиксель по одтельности, строить соответсвующие пикселю временные ряды изменений и искать на них "типичные" для вырубок изменения. Подход, по видимости, требует качественной фильтрации данных от мусора (облаков, теней) и уравнивания (стандартизации) снимков, сделанных в разное время (разное время года, разная освещенность, разные атмосферные условия). Представляется, что подобная предобработка данных будет довольно трудоемкой. *Текущий подход, который использует метрики, по сути является вариацией этого метода: метрики как раз и дают относительно устойчивые характеристики территории, которые сглаживают различия разновременных снимков.*
2. Анализ данных в пространственной области. Тут имеется в виду, что на снимках высокого разрешнения вырубки могут иметь характерный вид: центральная часть и идущие к ней волоки. Если это действительно так, то можно попробовать определять вырубки по подобным "отпечаткам". Это позволит избежать сложностей со стандартизацией разновременных снимков. С другой стороны, такой подход, вероятно, даст большое число ложных срабатываний на участках, которые похожи на вырубки (пролысины?, перекрестки дорог?). Тем не менее, мне представляется, что этот подход проще первого в реализации и, хотя первый путь потенциально более перспективен, я думаю, что нужно начать с поиска в пространственной области (легче реализовать => быстрее получим результаты => в случае удачи сэкономим время и усилия, а в случае неудачи потратимся не очень значительно).

Также возможны гибридные варианты, когда вырубка ищется и в пространственной, и во временной области.

Поэтому первая часть блокнота содержит раздел по визуализации имеющихся вырубок и визуального выделения признаков вырубок.

# Комментарий ИГ
1. Пока задача стоит разобратся с одним временем года (декабрь-март) и это с одной стороны проще - поскольку, нет фенологических изменений (за редким исключением), с другой стороны это сужает (возможно, на самом деле может быть и нет) возможности использования всяких индексов (NDVI и пр.). По первому подходу я вижу следующую последовательность действий (если это можно реализовать на данной платформе):

А) Создание базового безоблочного композита для зимы 2015 г. (если не получится набрать достаточное кол-во данных, добавить 2014). Здесь нам важно получить максимальное количество безоблачных пикселей и полное покрытие территории. Время отдельной сцены не важно.
Получение снимков и перевод в TOA - реализовано

QA анализ и нормализация - удаление облаков и теней на первом этапе можно использовать BQA слой, но думаю, что качество не будет нас устраивать, можно попробовать добавить буфера (элементы best pixels selection -http://www.tandfonline.com/doi/full/10.1080/07038992.2014.945827). 

Другие варианты - добавить FMASK (если это можно сделать на этой платформе) или сделать отдельный модуль для классификации на облака/воду/тени и т.д. Я могу сделать нужный слой треннингов для терриитории.

Про нормализацию: сначала, можно попробовать сделать вообще без нее (главное без облаков) и посмотреть что получится, но если возникнет необходимость добавлять данные (L7,S2), то понадобится. 

Если следовать "правильному" пути, то нужно сделать два модуля:

1) пересчитывает значения в каналах в зависимости от положения от центра снимка (можно просто взять коэф. регрессии из Петиной статьи, например, поскольку этот параметр вряд ли должен менятся в зав. от сезона) 
2) Нормализация по МОДИС (MOD43 или MOD09) -отбор "лучших" пикселей и пересчет (в среднее или медианное) значение за этот же период (2015) - построение ур. регрессии (MOD ~ LANDSAT) -коэф. использовать при обработке.

Создание композита.

Б) Обработка актуальных данных
Получение снимков и перевод в TOA - реализовано

QA анализ и нормализация - здесь пока можно ограничится только удалением облаков, поскольку изменения (рубки) в большинстве случаев сильно перекрывают всю прочую вариацию

Классификация

2. По второму подходу комментариев у меня пока нет

Планируется использовать -какие то текстурные характеристики брать с VHR и применять к Landsat -у?


## Подключение необходимых функций

Импортируем необходимые библиотеки и функции:

In [1]:
import utilites
reload(utilites)

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_location_path,
    get_wms_link,
)

In [2]:
from grasslib import GRASS

grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

Для того, чтобы не засорять PERMANENT вспомогательными и/или временными данными, переключимся в отдельный MAPSET (под названием exploration). Если такого набора нет, то он будет создан (флаг -c):

In [3]:
grs.grass.run_command('g.mapset', mapset='exploration', flags='c')
print grs.grass.read_command('g.mapset', flags='p')

exploration



## Как выглядят вырубки?

Для удобства отображения снимков воспользуемся WMS-сервисом. На снимки можно ссылаться следующим образом:

```
<img src="http://176.9.38.120/cruncher_wms/wms/landsat/?SERVICE=WMS&REQUEST=GetMap&BBOX=MinX,MinY,MaxX,MaxY&WIDTH=...&HEIGHT=...&LAYERS=...">
```

Например:
```
<img src="http://176.9.38.120/cruncher_wms/wms/landsat/?SERVICE=WMS&REQUEST=GetMap&BBOX=516684,5098286,543339.0,5113090.0&WIDTH=200&HEIGHT=200&LAYERS=toar_LC81120282015237LGN00_B5@landsat">
```


Снимок1 | Снимок2
--------|---------
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=516684,5098286,543339.0,5113090.0&WIDTH=200&HEIGHT=200&LAYERS=toar_LC81120282015237LGN00_B4@landsat">|<img src="http://176.9.38.120/cruncher_wms/wms/landsat/?SERVICE=WMS&REQUEST=GetMap&BBOX=516684,5098286,543339.0,5113090.0&WIDTH=200&HEIGHT=200&LAYERS=toar_LC81120282015237LGN00_B5@landsat">

Генерировать сылки можно при помощи вспомогательной функции:

In [18]:
print get_wms_link('toar_LC81120282015237LGN00_B5@landsat', 516684,5098286,543339.0,5113090.0)

http://176.9.38.120/cruncher_wms/wms/landsat/?SERVICE=WMS&REQUEST=GetMap&BBOX=516684,5098286,543339.0,5113090.0&WIDTH=300&HEIGHT=300&LAYERS=toar_LC81120282015237LGN00_B5@landsat


### Рубки в PlanetLabs

#### Вырубки, появившиеся зимой 2015-2016.

(В таблицах ниже ссылки в заголовках ведут на интерактивную карту для более детального рассмотрения.)

* Вырубка, появившаяся зимой в промежутке между 8-го января и 3-го февраля 2016-го года. Вырубка едва заметна. Найти ее можно по более светлому пятну по сравнению с окружающим его участком. Сам участок помимо того, что поменял цвет, стал более "шершавый" на вид. Размер вырубки порядка 100 метров в диаметре.

[До вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15057614.153513663&center=6043173.859893657&resolution=1.194328566955879&styles=16,29) | [После вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15057614.153513663&center=6043173.859893657&resolution=1.194328566955879&styles=16,23)
--------|---------
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=519572,5273372,520259,5273908&WIDTH=600&HEIGHT=600&LAYERS=20160108_011229_0c53_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=519572,5273372,520259,5273908&WIDTH=600&HEIGHT=600&LAYERS=20160302_225400_0c68_visual.composite@planet">



* Вырубка конца декабря 2015 года, размер приблизительно 50 метров по диагонали. Различия найти сложно, точнее на снимке от 7-го декабря также видна некоторая неоднородность структуры на фоне серой подстилки. (Возможно, ошибка в данных и вырубка случилась ранее декабря?). Вот еще ссылки на эту рубку, приведенные здесь для того, чтобы можно было оценить вариации условий съемки: [29.02.2016](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15055585.88502483&center=6033648.790990043&resolution=0.5971642834779395&styles=16,29), [03.03.2016](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15055585.88502483&center=6033648.790990043&resolution=0.5971642834779395&styles=16,23).

[До вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15055585.88502483&center=6033648.790990043&resolution=0.5971642834779395&styles=16,41) | [После вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15055585.88502483&center=6033648.790990043&resolution=0.5971642834779395&styles=16,35)
--------|---------
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=518184,5266924,518871,5267460&WIDTH=600&HEIGHT=600&LAYERS=20151207_013046_090c_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=518184,5266924,518871,5267460&WIDTH=600&HEIGHT=600&LAYERS=20151230_042202_0c07_visual.composite@planet">



* Вырубка конца декабря 2016 г. Ниже приводится снимок, сделанный до появления вырубки и два снимка, сделанные позднее. Каких-либо заметных отличий вырубки от окружающего фона не заметно (на месте вырубки присутствует "шершавое" пятно, но оно также присутствует и на снимке, сделанном до вырубки). Ошибка в описании данных?

[До вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15055920.894187862&center=6035699.453139506&resolution=1.194328566955879&styles=16,29) | [После вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15055920.894187862&center=6035699.453139506&resolution=1.194328566955879&styles=16,25) |  [После вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15055920.894187862&center=6035699.453139506&resolution=1.194328566955879&styles=16,23)
--------|---------|----
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=518347,5268331,519002,5268867&WIDTH=600&HEIGHT=600&LAYERS=20160108_011229_0c53_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=518347,5268331,519002,5268867&WIDTH=600&HEIGHT=600&LAYERS=20160229_040923_0c72_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=518347,5268331,519002,5268867&WIDTH=600&HEIGHT=600&LAYERS=20160302_225400_0c68_visual.composite@planet">

* Вырубка, помеченная как произошедшая в конце декабря. Но, по всей видимости, она случилась в январе ([ссылка](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15059061.381154671&center=6038739.616506692&resolution=1.194328566955879&styles=16,29) на январский снимок, на котором присутствует объект, похожий на вырубку).

[До вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15059061.381154671&center=6038739.616506692&resolution=1.194328566955879&styles=16,37) | [После вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15059061.381154671&center=6038739.616506692&resolution=1.194328566955879&styles=16,25)
--------|---------
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=520568,5270366,521223,5270901&WIDTH=600&HEIGHT=600&LAYERS=20151219_050318_0c53_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=520568,5270366,521223,5270901&WIDTH=600&HEIGHT=600&LAYERS=20160229_040923_0c72_visual.composite@planet">

#### Вырубки, появившиеся до конца осени 2015 г.

Ниже приводятся примеры вырубок, для которых нет данных PlanetLabs "до вырубки -- после вырубки", поэтому для них нельзя посмотреть направление изменений. Однако для этих примеров можно посмотреть, как выглядит вырубка на зимних снимках (почти) годом позднее. Поскольку вегетации в зимний период нет, то (ожидается), что на снимках представлено одно и то же состояние территории. Все различия, видимые на снимках, -- следствия различных условий съемки и состояния атмосферы. Поэтому эти примеры позволят оценить "внешнюю" вариативность данных, не связанную с реальными изменениями на местности.

* Вырубка, появившаяся в марте 2015 года. Ниже идут три зимних снимка одного и того же участка. 

[30.12.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15068502.728629654&center=6054030.424996231&resolution=1.194328566955879&styles=16,31) | [19.12.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15068502.728629654&center=6054030.424996231&resolution=1.194328566955879&styles=16,37) | [11.11.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15068502.728629654&center=6054030.424996231&resolution=1.194328566955879&styles=16,43)
--------|---------|----
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=526864,5280722,527551,5281257&WIDTH=400&HEIGHT=400&LAYERS=20151230_043745_0c47_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=526864,5280722,527551,5281257&WIDTH=400&HEIGHT=400&LAYERS=20151219_050318_0c53_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=526864,5280722,527551,5281257&WIDTH=400&HEIGHT=400&LAYERS=20151111_235840_0c64_visual.composite@planet">

* Аналогичный случай, участок на километр юго-восточнее: [30.12.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15069027.455881637&center=6053051.852888809&resolution=1.194328566955879&styles=16,31), [19.12.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15069081.200667148&center=6052904.9504750725&resolution=1.194328566955879&styles=16,37), [11.11.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15068980.695797933&center=6052987.171245538&resolution=1.194328566955879&styles=16,43).

* Снимки одной и той же вырубки, сделанные в разные дни. Почти на всех снимках вырубки видна дорога (волок?), проходящая в центральной части вырубки, а также лесная дорога:
 (А) Самый поздний снимок из имеющихся [02.03.2016-1](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,21), вырубка на нем выглядит как белое пятно на темном фоне. По текстуре она очень напоминает облако. Снимок сделанный в тот же день, но другой path/row [02.03.2016-2](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,23), каких-то видимых различий между ними нет, не смотря на то, что вырубка на снимке "а" находится на самом краю сцены. (Б) Снимок имеет более четко выраженную текстуру фона идующу в северо-северо-восточном направлении: [29.02.2016](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,25), по всей вероятности тот факт, что текстура видна не на всех снимках, каким-то образом связана с выпавшим (выпадающим?) снегом. (В) Очень похожая текстура присутствует на снимке [08.01.2016](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,29), но она расположена в северо-серверо-восточном направлении; эта разница в направлении текстур (ср. снимок Б) говорит об их временной природе (осадки в момент их перемещения или освещение). Мне представляется маловероятным, что языки снега, подобные видимым на снимках, связаны с его переносом (выпадение осадков или поземка), поскольку выпадение снега должно быть связано с облачностью, а поземка маловероятна(?) в лесу. (Г) Далее идут три снимка, сделанные в один день но разными path/row: [30.12.2016-1](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,31), [30.12.2016-2](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,33), [30.12.2016-3](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,35), на всех трех присутствует та же вытяннутая в одном и том же направлении текстура. Судя по теням от горных цепей снимки сделаны приблизительно в одно и то же время, поэтому они не позволяют проверить связано ли направление текстур с освещенностью.  (Д) [19.12.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,37), (Е) Снимки [07.12.2015-1](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,39), [07.12.2015-2](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,41) мало что добавляют уже известному. (Ж) Снимок [11.11.2015](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,43) является довольно мутным, поэтому несет не очень много информации. (З) Снимки [06.11.2015-1](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,45), [06.11.2015-2](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15062643.179391336&center=6038524.32818702&resolution=1.194328566955879&styles=16,47) интересны тем, что они сделаны до того, как на территории устоялся снежный покров. Эти снимки показывают, что текстура вырубки практически не видна на фоне леса, также лесные дороги, четко прослеживаемые на зимних снимках на основной части протяжения скрыты пологом леса. В частности, на этих снимках не видна дорога (волок?), пересекающая вырубку, заметную на зимних снимках.

### Общий вывод по PlanetLabs

Вырубки на зимних снимках выглядят очень похожими на облака: имеют ту же "шершавую" структуру и белый цвет. Они могут отличаться по текстуре от фона, но не всегда. Зачастую вырубки теряются на фоне других белых объектов, но часть из таких фоновых объектов можно отсеять по их линейной структуре: они занимают большие площади и их текстуры имеют характерные вытянутости вдоль одной из осей.

[До вырубки]() | [После вырубки]()
--------|---------
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=&WIDTH=600&HEIGHT=600&LAYERS=.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=&WIDTH=600&HEIGHT=600&LAYERS=.composite@planet">